In [7]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import datetime
from sklearn.preprocessing import LabelEncoder
warnings.simplefilter('ignore')
matplotlib.rcParams['figure.dpi'] = 100
sns.set()
%matplotlib inline

In [19]:
# Load Data
building = pd.read_csv('building_data_after_imputation_train.csv')
weather_train = pd.read_csv('weather_train.csv')
weather_test = pd.read_csv('weather_test.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [9]:
#Before we merge the dataset into one training dataset, we need to preprocessing the missing value first
#Function that plot missing value statistics
def missing_statistics(df):    
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns=['COLUMN NAME',"MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
    return statitics

#Function that fill missing value with timestamp method
def fill_missing_column(df,filler_df,col):
    null_df = df.loc[df[col].isnull()]
    
    if null_df.empty != True:
        null_df[col] = null_df.apply(lambda x: filler_df.loc[x['site_id']][x['day']][x['month']], axis=1)
        df.loc[null_df.index, col] = null_df[col]
    
    return df

In [10]:
#missing_statistics(building)

In [11]:
#missing_statistics(weather_train)

In [12]:
#missing_statistics(train)

In [20]:
def fill_weather_dataset(weather_df):
    
    # Find Missing Dates
    time_format = "%Y-%m-%d %H:%M:%S"
    start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
    end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

    missing_hours = []
    for site_id in range(16):
        site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_df = pd.concat([weather_df,new_rows])

        weather_df = weather_df.reset_index(drop=True)           

    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["day"] = weather_df["datetime"].dt.day
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])

    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)

    # Step 1
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

    weather_df.update(cloud_coverage_filler,overwrite=False)

    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)

    # Step 1
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
        
    return weather_df

weather_train = fill_weather_dataset(weather_train)
weather_test = fill_weather_dataset(weather_test)

In [14]:
#Below is the statistics table of missing value in each given dataset
missing_statistics(building)

In [15]:
missing_statistics(weather_train)

In [21]:
# Merge datasets into one train or test dataset
train = train.merge(building, on='building_id', how='left')
test = test.merge(building, on='building_id', how='left')

train = train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
test = test.merge(weather_test, on=['site_id', 'timestamp'], how='left')
del weather_train, weather_test,building
gc.collect()

# Encode dataset
le = LabelEncoder() 
train['primary_use'] = le.fit_transform(train['primary_use'])
test['primary_use'] = le.fit_transform(test['primary_use'])

In [22]:
missing_statistics(train)

,COLUMN NAME,MISSING VALUES,TOTAL ROWS,% MISSING
0,building_id,0,20216100,0.0
1,meter,0,20216100,0.0
2,timestamp,0,20216100,0.0
3,meter_reading,0,20216100,0.0
4,site_id,0,20216100,0.0
5,primary_use,0,20216100,0.0
6,square_feet,0,20216100,0.0
7,year_built,0,20216100,0.0
8,floor_count,0,20216100,0.0
9,air_temperature,0,20216100,0.0


In [23]:
missing_statistics(test)

,COLUMN NAME,MISSING VALUES,TOTAL ROWS,% MISSING
0,row_id,0,41697600,0.0
1,building_id,0,41697600,0.0
2,meter,0,41697600,0.0
3,timestamp,0,41697600,0.0
4,site_id,0,41697600,0.0
5,primary_use,0,41697600,0.0
6,square_feet,0,41697600,0.0
7,year_built,0,41697600,0.0
8,floor_count,0,41697600,0.0
9,air_temperature,0,41697600,0.0


In [26]:
export_csv = train.to_csv (r'new_train.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [27]:
export_csv = test.to_csv (r'new_test.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path